In [1]:
import torch
from torch import nn
from math import sqrt

class MultiHeadAttention(nn.Module):
  def __init__(self, input_dim, d_model, n_heads):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.n_heads = n_heads

    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    self.dense = nn.Linear(d_model, d_model)

    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    # 입력 x: (B, S, input_dim) - 배치 크기, 시퀀스 길이, 입력 차원
    print(f"입력 x shape: {x.shape}")
    
    # Linear 변환을 통한 Query, Key, Value 생성
    q, k, v = self.wq(x), self.wk(x), self.wv(x)  # 각각 (B, S, d_model)
    print(f"q, k, v shape: {q.shape}, {k.shape}, {v.shape}")
    
    B, S, D = q.shape[0], q.shape[1], self.d_model // self.n_heads
    print(f"B(배치 크기): {B}, S(시퀀스 길이): {S}, D(헤드별 차원): {D}")

    # 각 헤드별로 분할하기 위해 reshape 및 transpose
    q = q.reshape((B, S, self.n_heads, D)).transpose(1, 2)  # (B, n_heads, S, D)
    k = k.reshape((B, S, self.n_heads, D)).transpose(1, 2)  # (B, n_heads, S, D)
    v = v.reshape((B, S, self.n_heads, D)).transpose(1, 2)  # (B, n_heads, S, D)
    # q,k,v shape: torch.Size([2, 4, 5, 16]) 
    print(f"분할 후 q shape: {q.shape}")
    print(f"분할 후 k shape: {k.shape}")
    print(f"분할 후 v shape: {v.shape}")

    # 어텐션 스코어 계산
    score = torch.matmul(q, k.transpose(-1, -2))  # (B, n_heads, S, S)
    print(f"스코어 계산 후 shape: {score.shape}")
    
    # 스케일링
    score = score / sqrt(self.d_model)
    print(f"스케일링 후 score shape: {score.shape}")

    # 마스킹 적용 (필요한 경우)
    if mask is not None:
      print(f"마스크 shape: {mask.shape}")
      # mask: (B, S, 1) -> mask[:, None]: (B, 1, S, 1)
      

      # 마스크 shape: torch.Size([2, 5, 1])
      # 마스킹 후 score shape: torch.Size([2, 4, 5, 5])     

      score = score + (mask[:, None] * -1e9)
      print(f"마스킹 후 score shape: {score.shape}")

    # 소프트맥스 적용
    score = self.softmax(score)  # (B, n_heads, S, S)
    print(f"소프트맥스 후 score shape: {score.shape}")
    
    # Value와 가중합 계산
    result = torch.matmul(score, v)  # (B, n_heads, S, D)
    print(f"어텐션 결과 shape: {result.shape}")

    # 헤드 병합
    result = result.transpose(1, 2).reshape((B, S, -1))  # (B, S, d_model)
    print(f"헤드 병합 후 shape: {result.shape}")
    
    # 최종 출력층
    result = self.dense(result)  # (B, S, d_model)
    print(f"최종 출력 shape: {result.shape}")

    return result

In [2]:


# 테스트를 위한 MultiHeadAttention 클래스 (앞서 정의한 코드 사용)
# class MultiHeadAttention(nn.Module):
#     ...

# 테스트 파라미터 설정
batch_size = 4
seq_length = 10
input_dim = 32
d_model = 64
n_heads = 4

# 임의의 입력 텐서 생성
x = torch.randn(batch_size, seq_length, input_dim)
print(f"입력 텐서 생성: {x.shape}")

# 마스크 생성 (일부 위치를 패딩으로 표시)
# True는 패딩 토큰을 나타냄
mask = torch.zeros(batch_size, seq_length, 1, dtype=torch.bool)
for i in range(batch_size):
    # 각 배치마다 다른 길이의 패딩 설정 (예시)
    pad_length = torch.randint(0, seq_length//2, (1,)).item()
    mask[i, -pad_length:] = True
print(f"마스크 생성: {mask.shape}")
print(f"마스크 예시 (배치 0): {mask[0].squeeze()}")

# 모델 초기화
attention = MultiHeadAttention(input_dim, d_model, n_heads)
print(f"\n--- 모델 초기화 완료 ---")
print(f"입력 차원: {input_dim}, 모델 차원: {d_model}, 헤드 수: {n_heads}")

# 모델 실행
print(f"\n--- 모델 실행 ---")
output = attention(x, mask)

print(f"\n--- 결과 요약 ---")
print(f"입력 형태: {x.shape}")
print(f"출력 형태: {output.shape}")
print(f"각 헤드의 차원: {d_model//n_heads}")

# 첫 번째 배치의 첫 토큰 값 확인
print(f"\n--- 결과 샘플 ---")
print(f"출력 첫 번째 토큰: {output[0, 0, :5]}")  # 첫 5개 요소만 출력

입력 텐서 생성: torch.Size([4, 10, 32])
마스크 생성: torch.Size([4, 10, 1])
마스크 예시 (배치 0): tensor([False, False, False, False, False, False, False, False,  True,  True])

--- 모델 초기화 완료 ---
입력 차원: 32, 모델 차원: 64, 헤드 수: 4

--- 모델 실행 ---
입력 x shape: torch.Size([4, 10, 32])
q, k, v shape: torch.Size([4, 10, 64]), torch.Size([4, 10, 64]), torch.Size([4, 10, 64])
B(배치 크기): 4, S(시퀀스 길이): 10, D(헤드별 차원): 16
분할 후 q shape: torch.Size([4, 4, 10, 16])
분할 후 k shape: torch.Size([4, 4, 10, 16])
분할 후 v shape: torch.Size([4, 4, 10, 16])
스코어 계산 후 shape: torch.Size([4, 4, 10, 10])
스케일링 후 score shape: torch.Size([4, 4, 10, 10])
마스크 shape: torch.Size([4, 10, 1])
마스킹 후 score shape: torch.Size([4, 4, 10, 10])
소프트맥스 후 score shape: torch.Size([4, 4, 10, 10])
어텐션 결과 shape: torch.Size([4, 4, 10, 16])
헤드 병합 후 shape: torch.Size([4, 10, 64])
최종 출력 shape: torch.Size([4, 10, 64])

--- 결과 요약 ---
입력 형태: torch.Size([4, 10, 32])
출력 형태: torch.Size([4, 10, 64])
각 헤드의 차원: 16

--- 결과 샘플 ---
출력 첫 번째 토큰: tensor([-0.0995,  0.0652,  0.1418, 

In [3]:
# 다양한 입력 크기로 테스트
test_cases = [
    {"batch": 2, "seq": 5, "dim": 32},
    {"batch": 8, "seq": 20, "dim": 64},
    {"batch": 1, "seq": 100, "dim": 32}
]

print("\n=== 다양한 입력 크기 테스트 ===")
for i, case in enumerate(test_cases):
    b, s, d = case["batch"], case["seq"], case["dim"]
    print(f"\n테스트 케이스 {i+1}: 배치={b}, 시퀀스={s}, 차원={d}")
    
    # 입력 값
    
    x_test = torch.randn(b, s, d)
    mask_test = torch.zeros(b, s, 1, dtype=torch.bool)
    
    # 마스크 일부 설정
    for j in range(b):
        pad_len = s // 3  # 시퀀스의 1/3을 패딩으로 설정
        mask_test[j, -pad_len:] = True
    
    # 각 테스트 케이스마다 새 모델 생성
    test_attention = MultiHeadAttention(d, d_model, n_heads)
    output_test = test_attention(x_test, mask_test)
    
    print(f"  입력: {x_test.shape} -> 출력: {output_test.shape}")


=== 다양한 입력 크기 테스트 ===

테스트 케이스 1: 배치=2, 시퀀스=5, 차원=32
입력 x shape: torch.Size([2, 5, 32])
q, k, v shape: torch.Size([2, 5, 64]), torch.Size([2, 5, 64]), torch.Size([2, 5, 64])
B(배치 크기): 2, S(시퀀스 길이): 5, D(헤드별 차원): 16
분할 후 q shape: torch.Size([2, 4, 5, 16])
분할 후 k shape: torch.Size([2, 4, 5, 16])
분할 후 v shape: torch.Size([2, 4, 5, 16])
스코어 계산 후 shape: torch.Size([2, 4, 5, 5])
스케일링 후 score shape: torch.Size([2, 4, 5, 5])
마스크 shape: torch.Size([2, 5, 1])
마스킹 후 score shape: torch.Size([2, 4, 5, 5])
소프트맥스 후 score shape: torch.Size([2, 4, 5, 5])
어텐션 결과 shape: torch.Size([2, 4, 5, 16])
헤드 병합 후 shape: torch.Size([2, 5, 64])
최종 출력 shape: torch.Size([2, 5, 64])
  입력: torch.Size([2, 5, 32]) -> 출력: torch.Size([2, 5, 64])

테스트 케이스 2: 배치=8, 시퀀스=20, 차원=64
입력 x shape: torch.Size([8, 20, 64])
q, k, v shape: torch.Size([8, 20, 64]), torch.Size([8, 20, 64]), torch.Size([8, 20, 64])
B(배치 크기): 8, S(시퀀스 길이): 20, D(헤드별 차원): 16
분할 후 q shape: torch.Size([8, 4, 20, 16])
분할 후 k shape: torch.Size([8, 4, 20, 16])
분

<think>
질문은 마스크 테스트 값의 의미에 대해 묻고 있습니다. 코드에서 마스크를 어떻게 생성하고 사용하는지 분석해 봅시다.

코드에서 마스크 생성 부분:
```python
# 마스크 생성 (일부 위치를 패딩으로 표시)
# True는 패딩 토큰을 나타냄
mask = torch.zeros(batch_size, seq_length, 1, dtype=torch.bool)
for i in range(batch_size):
    # 각 배치마다 다른 길이의 패딩 설정 (예시)
    pad_length = torch.randint(0, seq_length//2, (1,)).item()
    mask[i, -pad_length:] = True
```

여기서 알 수 있는 점:
1. `mask`는 (batch_size, seq_length, 1) 형태의 boolean 텐서
2. 기본적으로 모든 값이 False로 초기화
3. 각 배치마다 다르게 시퀀스의 뒷부분 일부를 True로 설정
4. True는 패딩 토큰을 나타냄

그리고 MultiHeadAttention 클래스의 forward 메서드에서 마스크를 사용하는 부분:
```python
if mask is not None:
  score = score + (mask[:, None] * -1e9)
```

이것은 패딩 토큰(True)에 매우 작은 음수 값을 더해 소프트맥스 후에 해당 위치의 가중치가 0에 가까워지게 만듭니다.

따라서 마스크 테스트 값의 의미:
1. False: 실제 토큰 (어텐션 계산에 참여)
2. True: 패딩 토큰 (어텐션 계산에서 제외)
</think>

# 트랜스포머에서 마스크의 의미

멀티헤드 어텐션에서 사용된 마스크 테스트 값은 **패딩 토큰을 식별**하는 용도로 사용됩니다.

## 마스크 값의 의미

```python
# 테스트 코드에서 마스크 생성 부분
mask = torch.zeros(batch_size, seq_length, 1, dtype=torch.bool)
for i in range(batch_size):
    pad_length = torch.randint(0, seq_length//2, (1,)).item()
    mask[i, -pad_length:] = True
```

여기서:
- **False (0)**: 실제 의미 있는 토큰
- **True (1)**: 패딩 토큰 (무시해야 할 토큰)

## 마스크 사용 방법

```python
if mask is not None:
  score = score + (mask[:, None] * -1e9)
```

이 코드의 효과:
1. **패딩 토큰 위치**(True 값)에 매우 큰 음수 값(-1e9)을 더함
2. 소프트맥스 함수 적용 시 이 위치들은 0에 가까운 확률을 갖게 됨
3. 결과적으로 패딩 토큰은 어텐션 계산에 기여하지 않음

## 실제 응용 예시

1. **가변 길이 시퀀스 처리**:
   - 배치 처리를 위해 짧은 문장들은 패딩으로 채움
   - 패딩 토큰은 의미가 없으므로 마스크로 처리

2. **자기 회귀적(auto-regressive) 생성**:
   - GPT와 같은 모델에서는 미래 토큰을 보지 못하게 함
   - 이 경우 다른 형태의 마스크(Look-ahead 마스크)를 사용

3. **BERT의 양방향 어텐션**:
   - 패딩 마스크만 사용하여 양방향 문맥을 모두 고려

마스크는 트랜스포머 모델이 다양한 길이의 시퀀스를 효율적으로 처리하고, 특정 토큰을 무시할 수 있게 해주는 핵심 기술입니다.
